## Pre-processing

In [1]:

from llama_index.readers.file import PyMuPDFReader
from pathlib import Path
import config as cfg
from llama_index.core import Settings, VectorStoreIndex, StorageContext
from llama_index.core.storage.docstore import SimpleDocumentStore
from llama_index.vector_stores.faiss import FaissVectorStore
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core.retrievers import AutoMergingRetriever
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core import Document
import faiss
import os
from llama_index.llms.groq import Groq
import torch
from llama_index.core.node_parser import (
    HierarchicalNodeParser,
    SentenceSplitter,
)
from llama_index.core.node_parser import get_leaf_nodes, get_root_nodes

c:\Users\marco\projects\base\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def load_documents_from_directory(directory: Path) -> list:
    """Carica tutti i documenti PDF dalla directory specificata."""
    documents = []
    reader = PyMuPDFReader()
    
    # Itera su tutti i file PDF nella directory
    for file_path in directory.glob("*.pdf"):
        try:
            docs = reader.load_data(file_path)
            doc_text = "\n\n".join([d.get_content() for d in docs])
            doc_text = Document(text=doc_text)
            documents.append(doc_text)
            print(f"Caricato: {file_path.name}")
        except Exception as e:
            print(f"Errore nel caricamento di {file_path.name}: {e}")
    
    return documents

In [3]:


docs = load_documents_from_directory(cfg.DOCUMENTS_DIR)

Caricato: TICY-Linea guida tecniche di offuscamento dei dati e uso della crittografia.pdf
Caricato: TICY-Linee guida sullo sviluppo di software sicuro.pdf
Caricato: Tinexta Cyber-Vulnerability Management Policy.pdf
Caricato: Tinexta Cyber-Vulnerability Response Playbook.pdf
Caricato: TINEXTA CYBER_Piano di Risposta agli Incidenti - IRP.pdf


In [4]:


llm = Groq(model=cfg.LLM_MODEL,api_key=cfg.GROQ_API_KEY)

In [4]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
embed_model = HuggingFaceEmbedding(
    model_name="all-MiniLM-L6-v2",
    device=device
)


In [5]:
Settings.embed_model = embed_model


## Docling pre-processing

In [1]:
from docling.document_converter import DocumentConverter
from pathlib import Path
import config as cfg
from llama_index.core import Settings, VectorStoreIndex, StorageContext
from llama_index.core.storage.docstore import SimpleDocumentStore
from llama_index.vector_stores.faiss import FaissVectorStore
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core.retrievers import AutoMergingRetriever
from llama_index.core import Document
import faiss
import os
from llama_index.llms.groq import Groq
import torch
from llama_index.core.node_parser import (
    HierarchicalNodeParser,
)
from llama_index.core.node_parser import get_leaf_nodes, get_root_nodes

c:\Users\marco\projects\base\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from typing import List
def load_documents_from_directory(directory: Path) -> List[Document]:
    """
    Load and convert all PDF documents from specified directory using docling.
    
    Args:
        directory (Path): Path to directory containing PDF files
        
    Returns:
        List[Document]: List of converted documents
        
    Raises:
        Exception: If document loading or conversion fails
    """
    documents = []
    converter = DocumentConverter()
    
    # Iterate through all PDF files in directory
    for file_path in directory.glob("*.pdf"):
        try:
            # Convert PDF to text using docling
            doc_text = converter.convert(file_path)
            docling_text = doc_text.document.export_to_markdown()
            
            # Create Document object
            document = Document(text=docling_text)
            documents.append(document)
            
            print(f"Loaded: {file_path.name}")
            
        except Exception as e:
            print(f"Error loading {file_path.name}: {e}")
    
    return documents

In [3]:
docs = load_documents_from_directory(cfg.DOCUMENTS_DIR)

c:\Users\marco\projects\base\lib\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\marco\.cache\huggingface\hub\models--ds4sd--docling-models. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
c:\Users\marco\projects\base\lib\site-packages\torch\utils\cpp_extension.py:414: UserWarning: Error checking compiler v

Loaded: TICY-Linea guida tecniche di offuscamento dei dati e uso della crittografia.pdf
Loaded: TICY-Linee guida sullo sviluppo di software sicuro.pdf
Loaded: Tinexta Cyber-Vulnerability Management Policy.pdf
Loaded: Tinexta Cyber-Vulnerability Response Playbook.pdf
Loaded: TINEXTA CYBER_Piano di Risposta agli Incidenti - IRP.pdf


## Auto-merge retrieval (base)

In [14]:
node_parser = HierarchicalNodeParser.from_defaults(
    chunk_sizes=[1024, 8192],
    chunk_overlap=100, 
       
)
nodes = node_parser.get_nodes_from_documents(docs)


leaf_nodes = get_leaf_nodes(nodes)
root_nodes = get_root_nodes(nodes)

In [15]:
docstore = SimpleDocumentStore()

# insert nodes into docstore
docstore.add_documents(nodes)

In [24]:
dimension = len(embed_model.get_query_embedding("test"))
faiss_index = faiss.IndexFlatL2(dimension)
vector_store = FaissVectorStore(faiss_index=faiss_index)
    
    # Creiamo un contesto di archiviazione con il vector store
storage_context = StorageContext.from_defaults(
    vector_store=vector_store,
    docstore=docstore,
    )
    
    # Creazione dell'indice vettoriale
vector_index = VectorStoreIndex(
        leaf_nodes,
        storage_context=storage_context,
    )
    
    # Otteniamo il retriever base
base_retriever = vector_index.as_retriever(similarity_top_k=7)
    
    # Creiamo l'Auto Merging Retriever
auto_merging_retriever = AutoMergingRetriever(
        simple_ratio_thresh=0.5,
        vector_retriever=base_retriever,
        storage_context=storage_context,
        verbose=True
    )

In [26]:
from llama_index.core.response.notebook_utils import display_source_node
response = auto_merging_retriever.retrieve("Gli ambienti comunicano usando crittografia ?")
for node in response:
    display_source_node(node, source_length=10000)

> Merging 1 nodes into parent node.
> Parent node id: 4aab72d0-e651-47d3-bd78-403a7f71fe08.
> Parent node text: |
|                                            | occupa di gestione delle tecnologie.            ...

> Merging 1 nodes into parent node.
> Parent node id: 9f959064-9966-4f50-a3c3-a568b1dfda2d.
> Parent node text: 01

Pag. 1 di 51

Riservatezza:

Interno Tinexta Cyber SpA

<!-- image -->

Codice Procedura: PBV...

> Merging 1 nodes into parent node.
> Parent node id: d844d0de-bdff-4dd3-b0a5-fc3668c99d2a.
> Parent node text: ·                                                                                                ...

> Merging 1 nodes into parent node.
> Parent node id: 72c1bbf4-b250-48b8-bac8-14d4506c7fe6.
> Parent node text: - · L'obiettivo è motivare il team e spingerlo a impiegare prassi e pattern appropriati, aderendo...

> Merging 1 nodes into parent node.
> Parent node id: 2b625d70-c665-49c6-8ef0-ec0b93fbd82b.
> Parent node text: 31 di 71

Riservatezza: Interno 

**Node ID:** 978dfc55-6c5b-49e3-b6d7-fbc576395d08<br>**Similarity:** 1.0487700700759888<br>**Text:** 03

Pag. 118 di 156

Riservatezza:

Interno Tinexta Cyber SpA

SGI ISO 9001/ISO 27001

| Gestione configurazioni   | · la gestione delle configurazioni deve permettere e agevolare il più possibile patching  e fixing tempestivo.                                                                                                                                                                                                                                                         |
|---------------------------|------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| Gestione errori           | · intercettazione di tutti i possibili errori, in modo che siano gestiti, la presenza di errori  non gestiti è vietata;                                                                                                                                                                                                                                                |
|                           | · quando  un'operazione  dà  errore  l'utente  dovrebbe  essere  avvertito  e  tutte  le  operazioni successive dovrebbero essere bloccate -  fail safe ; eventuali divergenze da  questa regola devono essere motivate, documentate e tracciate.                                                                                                                      |
|                           | · l'avvertimento  all'utente  dev'essere  di  alto  livello  e  non  deve  rivelare  dati  o  informazioni  che  potrebbero  essere  usate  da  un  attaccante  - fail  secure ;  es.  non  devono essere visualizzati  stacktrace  o altri messaggi contenenti dettagli sulle cause  dell'errore, sull'architettura dell'applicazione o sulle tecnologie sottostanti. |

## 5.4.4.2 Requisiti di codifica sicura

Dipendono dal linguaggio utilizzato e devono essere descritti in documenti specifici, o anche in wiki a uso interno  o  in  regole  di  sviluppo  sicuro  che,  per  il  team,  recepiscono  queste  linee  guida,  o  anche,  se  la metodologia è Agile, nelle Definition of Done .

Tabella 10 -Esempi di requisiti di codifica

| Requisito Descrizione                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              |
|------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| · standard  per  denominare  classi,  metodi,  variabili  e  costanti  in  modo  che  ne  sia                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      |
| Convenzione dei nomi  comprensibile la finalità e la natura.  Commenti  · gestione dei commenti, quantità, collocazione;  · se i commenti sono troppo presenti e verbosi, è un indizio che il codice non è auto- esplicativo, pertanto ne andrebbero riesaminati i requisiti relativi all'architettura; · attenzione  affinché  nei  commenti  non  siano  esposte  informazioni  utili  per  comprendere  l'architettura  o  altre  informazioni  critiche,  per  le  quali  è  necessario  valutare se procedere a un offuscamento (es. offuscamento del codice Javascript).  · gestione  delle  variabili  per  prevenire  gli  attacchi  più  diffusi  (es.  esternalizzazione |
| Variabili                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |
| Funzioni / istruzioni da  evitare  · anche se consentite dal linguaggio di programmazione, censire le funzioni / istruzioni  da evitare o da tenere sotto controllo, anche con strumenti automatici di analisi (es.                                                                                                                                                                                                                                                                                                                                                                                                                                                                |

## 5.4.5 Superficie di attacco e valutazione del rischio

Occorre  essere  consapevoli  che  ogni  funzionalità  introdotta  aggiunge  una  certa  quantità  di  rischio all'applicazione nel suo complesso.

L'obiettivo dello sviluppo sicuro è ridurre il rischio complessivo minimizzando la 'superficie di attacco', un concetto  comunemente  applicato  per  migliorare  la  sicurezza  dei  sistemi  software,  essendo  utile  nella valutazione del rischio e nella definizione di misure volte non solo a eliminarlo, ma anche a mitigare gli effetti derivanti da un attacco riuscito.<br>

**Node ID:** dcda9c21-4334-4dc4-bb96-51f96c1ca8af<br>**Similarity:** 1.0485237836837769<br>**Text:** <!-- image -->

## Linee guida tecniche di offuscamento dei dati e uso della crittografia

Codice Procedura: LGTODUC

Rev. 01

Pag. 20 di 71

Riservatezza: Interno Tinexta Cyber SpA

ISO/IEC 27001

Fig. 5 - Risultato di un processo di pseudonimizzazione che ha sostituito nella prima tabella il nome con un token (codice)

| Nome    | Sesso   |   Anno Nascita |    CAP | DIAGNOSI   |   Token | Sesso   |   Anno Nascita |    CAP | DIAGNOSI   |
|---------|---------|----------------|--------|------------|---------|---------|----------------|--------|------------|
| Verdi   |         |           1962 | 300122 | Cancro     |   11779 |         |           1962 | 300122 | Cancro     |
| Rossi   |         |           1961 | 300133 | Gastrite   |   12121 |         |           1961 | 300133 | Gastrite   |
| Gialli  | M       |           1950 | 300111 | Inlarto    |   21177 | M       |           1950 | 300111 | Infarto    |
| Neri    | M       |           1954 | 300112 | Emicrania  |   41898 | M       |           1954 | 300112 | Emicrania  |
| Bianchi |         |           1965 | 300200 | Lussazione |   56789 |         |           1965 | 300200 | Lussazione |
| Rosa    | M       |           1953 | 300115 | Frattura   |   65656 | M       |           1953 | 300115 | Fraltura   |

## 2.2.2.2 Funzione di hash

Corrisponde a una funzione che, a partire da un input di dati di qualsiasi dimensione (l'immissione potrebbe essere  costituita  da  un  unico  attributo  o  da  un  insieme  di  attributi),  restituisce  comunque  un  output  di dimensione fissa. Tale funzione non può essere invertita, vale a dire che non esiste il rischio di inversione associato alla crittografia; t uttavia, se l'intervallo dei valori di immissione relativi alla funzione di hash è noto, la funzione stessa consente di riprodurli al fine di desumere il valore corretto associato a un dato specifico.

Le funzioni di hash sono solitamente progettate in modo da procedere con calcoli rapidi e sono soggette ad attacchi brute-force. Ad esempio, se un insieme di dati è stato pseudonimizzato effettuando l'hashing del codice fiscale, lo stesso, teoricamente, può essere estrapolato effettuando l'hashing di tutti i possibili valori di input e raffrontando il risultato con i valori contenu ti nell'insieme di dati.

Si possono inoltre creare tabelle precalcolate per consentire l'inversione in blocco di un insieme consistente di valori hash. Il ricorso a una funzione di hash con salt (che prevede l'aggiunta di un valore casuale, noto come 'salt',  all'attributo  oggetto di  hashing)  può  ridurre  la  probabilità  di  estrapolare  il  valore  di  input, permanendo tuttavia la possibilità di calcolare con mezzi ragionevolmente utilizzabili il valore dell'attributo originario nascosto dietro al risultato di una funzione di hash con salt.

## 2.2.2.3 Funzione di hash cifrato con chiave memorizzata

Corrisponde a una funzione di hash particolare che utilizza una chiave segreta quale  input aggiuntivo (la differenza rispetto alla funzione di hash con salt è che il salt abitualmente non è segreto).<br>

**Node ID:** 2b625d70-c665-49c6-8ef0-ec0b93fbd82b<br>**Similarity:** 1.0276541709899902<br>**Text:** 31 di 71

Riservatezza: Interno Tinexta Cyber SpA

ISO/IEC 27001

Fig. 10 - Crittografia ibrida, HTTPS handshaking

<!-- image -->

## 2.2.5.5.4 Vantaggi e svantaggi

In Tabella 5 sono riassunti vantaggi e svantaggi dei tre principali tipi di sistemi crittografici.

Tabella 5 -Tipi di crittografia, vantaggi e svantaggi

| Tipo di  Crittografia   | Vantaggi                                                | Svantaggi                                                                        |
|-------------------------|---------------------------------------------------------|----------------------------------------------------------------------------------|
| Simmetrica              | Molto veloce                                            | · Numero di chiavi molto elevato  · Problema dello scambio delle chiavi  ·       |
|                         | Non serve un canale sicuro per lo scambio  delle chiavi | Asimmetrica  Molto  lenta,  a  causa  dei  calcoli  complessi  da  effettuare  · |
| Ibrida                  | Coniuga i vantaggi di entrambe le tecniche,  -          |                                                                                  |

<!-- image -->

## Linee guida tecniche di offuscamento dei dati e uso della crittografia

scambio delle chiavi condivise è utilizzato il cifrario asimmetrico;

- · sicurezza della crittografia asimmetrica, che però è usata solo una volta, superando il limite della lentezza di elaborazione, poiché per la comunicazione è utilizzato il cifrario simmetrico.

Guardando alla Tabella 5, è importante sottolineare come la crittografia asimmetrica non sia contrapposta a quella  simmetrica,  bensì  la  integri  e  la  complementi,  permettendo  a  entrambe  di  svolgere  un  ruolo determinante nella gestione e nello scambio di chiavi sicuro anche sulla rete Internet, canale notoriamente non sicuro, ossia normalmente non crittografato e potenzialmente accessibile a tutti.

## 2.2.5.5.5 Quando utilizzarle

Viste le osservazioni al § precedente, in Tabella 6 è riassunto quando è utile utilizzare ciascuna delle tecniche di crittografia menzionate.

Tabella 6 - Tipi di crittografia, quando utilizzarli

| Tipo di  Crittografia   | Quando utilizzarlo                                                                                                                                                                                                                                                                                                                                                                                                                                                   |
|-------------------------|----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| Simmetrica              | · Prestazioni elevate: è generalmente più veloce e computazionalmente meno intensiva rispetto  alla crittografia asimmetrica; è ideale per cifrare grandi quantità di dati.  · Canali di comunicazione sicuri: in presenza di un canale sicuro per scambiare la chiave segreta tra  le parti, la crittografia simmetrica è una buona scelta.  · Memorizzazione dati: è spesso usata per proteggere dati a riposo, come file su un disco rigido,  database, o backup. |
| Asimmetrica             | · Distribuzione delle chiavi: quando non è possibile utilizzare un canale sicuro per distribuire chiavi  segrete, la crittografia asimmetrica è utile; consente la distribuzione sicura delle chiavi pubbliche,  che possono essere utilizzate per cifrare dati che solo il proprietario della chiave privata può  decifrare.                                                                                                                                        |
|                         | · Autenticazione: la crittografia asimmetrica è ideale per la verifica delle identità tramite firme  digitali e certificati.  · Integrazione  e  verifica:  è  utilizzata  per  garantire  l'integrità  e  l'autenticità  dei  messaggi,  ad                                                                                                                                                                                                                         |
| Ibrida                  | · Comunicazione sicura: sfruttando i vantaggi di entrambi i tipi di crittografia, utilizza la crittografia  asimmetrica per scambiare in modo sicuro una chiave di sessione, che poi viene utilizzata per la  crittografia simmetrica dei dati effettivi.<br>

**Node ID:** 72c1bbf4-b250-48b8-bac8-14d4506c7fe6<br>**Similarity:** 1.0084837675094604<br>**Text:** - · L'obiettivo è motivare il team e spingerlo a impiegare prassi e pattern appropriati, aderendo ai modelli concettuali Clean Code e Clean Architecture, incoraggiando una cultura che non accetti pratiche di codifica autonome, complicate e confuse, e come tali insicure.

## 3.3.3 Misure di sicurezza da applicare

## 3.3.3.1 Criteri di verifica e checklist

| 4.1.1   | Autenticazione, Autorizzazione e Gestione degli accessi   |
|---------|-----------------------------------------------------------|
| 4.1.2   | Schemi di sicurezza e crittografia                        |
| 4.1.3   | Performance minime                                        |
| 4.1.4   | Input data validation                                     |
| 4.1.5   | Tracciamento Eventi e Alarm Detection                     |
| 4.1.6   | Sviluppo, compilazione e test  dell'applicazione          |
| 4.1.7   | Altre misure di sicurezza                                 |
| 4.2.1   | ASVS, gruppi di controlli                                 |
| 4.2.2   | MASVS, gruppi di controlli                                |
| 4.2.3   | Top 10 Proactive Controls                                 |
| 4.2.4   | Top 10 Web                                                |
| 4.2.5   | Top 10 CI/CD Security                                     |
| 4.2.6   | Mobile Top 10                                             |
| 4.2.7   | API Security Top 10                                       |
| 4.2.8   | Cloud-Native Application Security Top 10                  |
| 4.2.9   | Docker Top 10                                             |
| 4.2.10  | Kubernetes Top 10                                         |
| 4.2.11  | Desktop App Security Top 10                               |
| 4.2.12  | Low-Code/No-Code Top 10                                   |

## Linee guida sullo sviluppo di software sicuro

Codice Procedura: LGSWSIC

Rev. 03

Pag. 37 di 156

Riservatezza:

Interno Tinexta Cyber SpA

SGI ISO 9001/ISO 27001

devono essere ripetuti a un intervallo prestabilito (es. 24h) e l'intervallo deve essere specificato nella documentazione di progetto.

- · Nella documentazione di progetto devono essere indicate anche le impostazioni selezionate per il 'quality gate', es.
- o Coverage > 80%
- o Duplicated lines: <3
- o Reliability rating: A
- o Security ratying: A
- · In caso di problemi rilevati nelle scansioni effettuate il 'quality gate' deve interrompere la build e impedire la release e il deployment del codice, notificando al team di sviluppo quanto riscontrato.
- · In primo luogo, è essenziale che il team stabilisca quali peer review effettuare e come, es.:
- o stato di un item (una singola classe, un singolo componente, un'API completa, ecc.);
- o task specifici nello strumento di tracking del team;
- o pull request nel repository del codice.
- · I risultati della revisione e i suggerimenti apportati devono essere allegati alla documentazione di progetto, per guidare le attività successive, es. annotando i risultati della peer review nello strumento di tracking del team, nel task specifico oggetto della revisione; o anche, se il team è Agile, per valutare il debito tecnico nella review dello Sprint in corso e la definizione della quantità di lavoro per lo Sprint successivo.
- · Le revisioni paritetiche sono essenziali nella review del codice realizzato, ma dovrebbero essere presenti, con modalità simili, in ogni momento e fase, dall'inizio alla fine del progetto, analisi, disegno, sviluppo, test, ecc.<br>

**Node ID:** d844d0de-bdff-4dd3-b0a5-fc3668c99d2a<br>**Similarity:** 1.0047482252120972<br>**Text:** ·                                                                                                                                                                                                         |

## 2.2.5.6 Tecniche di crittoanalisi

Le tecniche di crittoanalisi si riferiscono non solo ai metodi per rompere un cifrario, ma anche a ogni tentativo di  eludere  la  sicurezza  di  algoritmi  e  protocolli  crittografici,  ad  esempio  mediante  un  attacco  al  canale laterale.

## 2.2.5.6.1 Principali tecniche di crittoanalisi

Codice Procedura: LGTODUC

Rev. 01

Pag. 32 di 71

Riservatezza:

Interno Tinexta Cyber SpA

ISO/IEC 27001

<!-- image -->

## Linee guida tecniche di offuscamento dei dati e uso della crittografia

Codice Procedura: LGTODUC

Rev. 01

Pag. 33 di 71

Riservatezza:

Interno Tinexta Cyber SpA

ISO/IEC 27001

Esistono diverse tecniche di crittoanalisi, le più famose sono:

- · Forza bruta (brute-force attack), ossia un attacco finalizzato a ricreare i dati di input tramite il calcolo di tutte le possibili combinazioni di chiavi del cifrario. Con l'aumento della potenza di calcolo questa tecnica diventa sempre più efficace.
- · Crittoanalisi differenziale , tecnica a testo in chiaro scelto. Si crittano due testi in chiaro con differenze note e si analizza come tali differenze si propagano fino all'output, alla ricerca di eventuali regolarità (in un buon algoritmo invece la relazione tra testo in chiaro e testo cifrato è sempre apparentemente casuale, cfr. Diffusione e Confusione in § ' 2.2.5.2 Altre definizioni importanti ' ).
- · Crittoanalisi lineare , si cercano delle funzioni lineari che mettano in relazione testo in chiaro, testo crittato e chiave.
- · Man-in-the-middle , sfruttando il sistema delle infrastrutture a chiavi pubbliche un eventuale intruso può posizionarsi tra un mittente ed un destinatario e scambiare le loro chiavi pubbliche e private con altre opportunamente modificate.

Esempi:  DES  può  essere  violato  conoscendo  243  testi  in  chiaro  scelti,  mediante  tecniche  di  crittoanalisi differenziale, o 241 testi in chiaro, mediante tecniche di crittoanalisi lineare.

## 2.2.5.6.2 Scenari

Le tecniche si applicano ai seguenti scenari:

- · s olo testo cifrato, quando l'attaccante ha accesso solo ad una collezione di testi cifrati;
- · t esto  in  chiaro  noto,  quando  l'attaccante  ha  un  insieme  di  testi  cifrati  dei  quali  conosce  i corrispondenti testi in chiaro;
- · t esto in chiaro scelto, quando l'attaccante può ottenere i testi cifrati corrispondenti ad un insieme arbitrario di testi in chiaro di sua scelta;
- · a ttacco alle chiavi correlate, quando l'attaccante può ottenere uno stesso testo in chiaro cifrato con due differenti chiavi; le chiavi sono ignote ma la relazione fra esse è nota, ad esempio due chiavi che differiscono di 1 solo bit.<br>

**Node ID:** 9f959064-9966-4f50-a3c3-a568b1dfda2d<br>**Similarity:** 0.9997286796569824<br>**Text:** 01

Pag. 1 di 51

Riservatezza:

Interno Tinexta Cyber SpA

<!-- image -->

Codice Procedura: PBVRESP

Rev. 01

Pag. 2 di 51

Riservatezza: Interno Tinexta Cyber SpA

## Sommario

| Obiettivi e campo di applicazione .........................................................................................................4                | Obiettivi e campo di applicazione .........................................................................................................4                |
|-------------------------------------------------------------------------------------------------------------------------------------------------------------|-------------------------------------------------------------------------------------------------------------------------------------------------------------|
| 1.1                                                                                                                                                         | Introduzione ..................................................................................................................................4            |
| 1.2                                                                                                                                                         | Obiettivi .........................................................................................................................................4        |
| 1.3                                                                                                                                                         | Campo di applicazione ...................................................................................................................4                  |
| 1.4                                                                                                                                                         | Riferimenti .....................................................................................................................................5          |
| 1.5                                                                                                                                                         | Glossario dei Termini .....................................................................................................................7                |
| 2  Risposta alle Vulnerabilità .................................................................................................................. 16        | 2  Risposta alle Vulnerabilità .................................................................................................................. 16        |
| 2.1                                                                                                                                                         | Definizioni ................................................................................................................................... 16          |
| 2.1.1                                                                                                                                                       | Segnalazioni, Eventi, Incidenti e Vulnerabilità ................................................................... 16                                      |
| 2.1.2                                                                                                                                                       | Rapidità di risposta e sua importanza ................................................................................ 17                                   |
| 3  Processo di Vulnerability Response .................................................................................................... 18               | 3  Processo di Vulnerability Response .................................................................................................... 18               |
| 3.1  Aree aziendali e Ruoli coinvolti .................................................................................................. 18                 | 3.1  Aree aziendali e Ruoli coinvolti .................................................................................................. 18                 |
| 3.2  Fasi .............................................................................................................................................. 19 | 3.2  Fasi .............................................................................................................................................. 19 |
| 3.2.1                                                                                                                                                       | Preparazione ....................................................................................................................... 19                     |
| 3.2.2                                                                                                                                                       | Identificazione .................................................................................................................... 19                     |
| 3.2.3                                                                                                                                                       | Valutazione e decisione ...................................................................................................... 20                           |
| 3.2.4                                                                                                                                                       | Risoluzione .......................................................................................................................... 21                   |
| 3.2.5                                                                                                                                                       | Segnalazione e Notifica ...................................................................................................... 23                           |
| 3.3  Workflow .................................................................................................................................... 23       | 3.3  Workflow .................................................................................................................................... 23       |
| 3.3.1                                                                                                                                                       | Asset Tinexta Cyber, mantenuti da Tinexta Cyber ............................................................. 24                                            |
| 3.3.2                                                                                                                                                       | Asset Tinexta Cyber, mantenuti da Fornitori di Tinexta Cyber........................................... 25                                                  |
| 3.4                                                                                                                                                         | Matrice RACI ............................................................................................................................... 26             |
| 3.5                                                                                                                                                         | Strumenti .................................................................................................................................... 28           |
| 3.5.1                                                                                                                                                       | Acquisizione di informazioni sulle vulnerabilità ................................................................. 28                                       |
| 3.5.2                                                                                                                                                       | Tracciamento delle attività ................................................................................................. 29                            |
| 3.5.3                                                                                                                                                       | Condivisione di valutazioni e correzioni intraprese ............................................................ 31                                         |
| 3.5.4                                                                                                                                                       | Misurazione delle Vulnerabilità e Valutazione delle Minacce ........................................... 32                                                  |
| 3.6  Comunicazioni ............................................................................................................................ 34          | 3.6  Comunicazioni ............................................................................................................................ 34          |
| Appendice ........................................................................................................................................... 35    | Appendice ........................................................................................................................................... 35    |
| 4.1  Appendice A: Descrizione Ruoli coinvolti ................................................................................... 35                        | 4.1  Appendice A: Descrizione Ruoli coinvolti ..........................................................................<br>

**Node ID:** 4aab72d0-e651-47d3-bd78-403a7f71fe08<br>**Similarity:** 0.9884227514266968<br>**Text:** |
|                                            | occupa di gestione delle tecnologie.                                                                                                                                                                                                                                                                                                                                               |

## Linee guida sullo sviluppo di software sicuro

<!-- image -->

Codice Procedura: LGSWSIC

Rev. 03

Pag. 17 di 156

Riservatezza:

Interno Tinexta Cyber SpA

SGI ISO 9001/ISO 27001

| Termine           | Descrizione                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   |
|-------------------|-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| OECD              | Organization for Economic Co-operation  and Development, è un'Organizzazione interna - zionale  che  opera  per  creare  politiche  migliori  per  una  vita  migliore ,  con  l' obiettivo  di  promuovere politiche che favoriscano prosperità, uguaglianza, opportunità e benessere  per tutti.                                                                                                                                                                                                                                                                            |
| Offuscatore       | Software  che  converte  il  codice  sorgente  in  forma  difficilmente  interpretabile  o  non  interpretabile del tutto al fine di inibire l'utilizzo di tecniche di reverse engineering.                                                                                                                                                                                                                                                                                                                                                                                   |
| OOD               | Object Oriented Design, un approccio alla progettazione dove, allo scopo di descrivere un  software,  il  sistema  è  rappresentato  mediante  oggetti  intesi  come  entità/concetti  del  dominio  del  problema  con  un  proprio  stato  (struttura  dei  dati)  e  comportamento  (operazioni/funzioni), capaci al contempo di memorizzare ed elaborare informazioni e in  grado di interagire gli uni con gli altri attraverso lo scambio di messaggi. A tale approccio si  riferisce anche l'espressione paradigma orientato agli a oggetti , o  paradigma a oggetti . |
| Organizzazione    | In questo contesto ci rifieriamo a imprese, aziende, gruppi societari, società, divisioni, enti  locali o centrali della Pubblica Amministrazione, studi professionali, associazioni, ecc., con  accezione più ampia rispetto al termine "organizzazione" in senso stretto (le funzioni, i  livelli e i collegamenti, le responsabilità, le regole, l'articolazione dimensionale, ecc.).                                                                                                                                                                                      |
| OSCE              | Organizzazione per la Cooperazione e lo Sviluppo Economico, cfr. OECD.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        |
| OWASP             | Open Web Application Security Project, nato nel 2001 con lo scopo di diffondere la cultura  della qualità del codice nelle fasi di sviluppo del software (Web). Opera secondo un modello  "aperto" tipico dell' open-source ,  il  che significa che chiunque può intervenire nelle chat,  contribuire a progetti e altro relativi a OWASP.                                                                                                                                                                                                                                   |
| Parti interessate | Cfr. PT.  Penetration Test, o anche Pen Test, o Pentesting, è una tecnica di sicurezza informatica                                                                                                                                                                                                                                                                                                                                                                                                                                                                            |
|                   | Cfr. Stakeholders.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            |
| Penetration test  |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               |
| PT                |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               |
| PBI               | Product Backlog Item, un elemento del Product Backlog.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        |
| Product Backlog   | In Agile, è una lista di funzionalità in continua evoluzione.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 |

## Linee guida sullo sviluppo di software sicuro

<!-- image -->

Codice Procedura: LGSWSIC

Rev. 03

Pag. 18 di 156

Riservatezza:

Interno Tinexta Cyber SpA

SGI ISO 9001/ISO 27001

| Termine             | Descrizione                                                                                                                                                                                                                                                                                                                                                                                                                                                           |
|---------------------|-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| Rainbow Table       | Tabella di associazione usata per il recupero di testo in chiaro partendo...<br>

## Query expansion

In [ ]:
from llama_index.core.postprocessor import SentenceTransformerRerank
from llama_index.core.indices.query.query_transform.base import StepDecomposeQueryTransform
from llama_index.core.query_engine import MultiStepQueryEngine
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.prompts import PromptTemplate

In [ ]:
Settings.llm = llm

In [ ]:
DIRECT_QUERY_TEMPLATE = """
Sei un assistente di ricerca che aiuta a decomporre domande complesse in sottodomande più semplici in italiano. Ogni sottodomanda deve includere informazioni contenute nella domanda originale.

Contesto sulla fonte di conoscenza:
{index_summary}

Domanda originale: {query}

La tua risposta deve contenere SOLO una sottodomanda specifica in italiano, senza spiegazioni o ragionamenti. 
Non includere frasi come "Ecco una sottodomanda" o "La sottodomanda è".
Rispondi con una singola frase interrogativa.
"""

In [ ]:
step_decompose_transform = StepDecomposeQueryTransform(
    llm=llm,
    verbose=True,
    #step_decompose_query_prompt=PromptTemplate(DIRECT_QUERY_TEMPLATE),
    )

In [ ]:


# Crea il query engine direttamente con il tuo retriever
base_query_engine = RetrieverQueryEngine.from_args(
    retriever=auto_merging_retriever,
    llm=llm
)
index_summary = "Questo indice contiene documenti PDF relativi a policy aziendali di vario tipo. Gli articoli sono stati caricati da un dataset di documenti PDF e indicizzati con un indice vettoriale. Il motore di ricerca è in grado di rispondere a domande relative ai contenuti dei documenti."

In [ ]:

query_engine = MultiStepQueryEngine(
    query_engine=base_query_engine,
    query_transform=step_decompose_transform,
    index_summary=index_summary
)

response = query_engine.query('''È stato stabilito, documentato, approvato, comunicato, applicato, valutato e mantenuto un piano d'azione correttivo basato sul rischio per rimediare ai risultati dell'audit?''')
print(response)


## Keyword extraction for metadata

### KeyBert

In [ ]:
from keybert import KeyBERT
import torch
from typing import List
from sentence_transformers import SentenceTransformer

# Controlla se è disponibile la GPU
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Dispositivo in uso: {device}")

# Inizializza il modello Sentence Transformer con GPU (KeyBERT usa questo sotto il cofano)
model_name = "all-mpnet-base-v2"  # Questo modello è multilingue e leggero
sentence_model = SentenceTransformer(model_name, device=device)

if device == "cuda":
    print(f"Utilizzando la GPU: {torch.cuda.get_device_name(0)}")

# Inizializza KeyBERT con il modello precaricato su GPU
kw_model = KeyBERT(model=sentence_model)

# Esempio di chunk di testo
chunk = """
Gli audit log devono registrare il verificarsi di un evento, catturando informazioni come l'ora in cui si è verificato, l'utente o il servizio responsabile, es.: o identificativo, descrizione e timestamp dell'evento; o attore (es. ID utente o ID API) e azione eseguita (creazione, modifica, cancellazione, ecc.; o elemento impattato (es. indirizzo IP e identificativo dell’applicazione, dispositivo, sistema, ecc.); o fonte da cui proviene l'attore (paese, nome host, indirizzo IP, ID dispositivo, ecc.); o eventuali tag personalizzati, come il livello di gravità attribuito all'evento (es. clear, warning, minor, major, critical). Asset Management • Onde scongiurare fenomeni di “Shadow IT”, deve essere garantito e documentato l’inserimento, nel’Asset Inventory aziendale, dell’applicazione e dell’ambiente in cui essa opera, di produzione o meno. • L’inserimento nell’Asset Inventory aziendale deve includere il nominativo di chi ha la titolarità (ownership) dell’applicazione / ambiente, ovvero della funzione o ruolo che ha la responsabilità primaria dell’attività, della produttività e della resilienza dell’asset in questione. • Questo principio è essenziale per la risposta alle vulnerabilità e la gestione continua delle stesse, dato che il processo aziendale di risposta a vulnerabilità presenti su un determinato “Asset” risulterebbe bloccato nel caso non fosse preventivamente individuato il relativo “Asset Owner”, cfr. [41].

Linee guida
sullo sviluppo di software sicuro Codice Procedura: LGSWSIC Rev. 03 Pag.
"""

# Estrai parole chiave direttamente
keywords = kw_model.extract_keywords(
    chunk,
    keyphrase_ngram_range=(1, 2),  # Considerare unigrammi e bigrammi          # KeyBERT gestisce automaticamente anche per testi in italiano
    use_mmr=True,                  # Usa Maximal Marginal Relevance per diversificare
    diversity=0.7,                 # Bilanciamento tra rilevanza e diversità
    top_n=3                       # Numero di parole chiave da estrarre
)

# Stampa le parole chiave estratte con i loro punteggi
print("\nParole chiave estratte con punteggi:")
for keyword, score in keywords:
    print(f"- {keyword}: {score:.4f}")

# Estrai solo le parole chiave (senza punteggi)
keyword_list = [keyword for keyword, _ in keywords]

# Crea un Document di LlamaIndex con le parole chiave nei metadati
document = Document(
    text=chunk,
    doc_id="doc1",
    metadata={"keywords": keyword_list}
)

print(f"\nDocument creato con ID: {document.doc_id}")
print(f"Metadati con parole chiave: {document.metadata}")

### SpaCy

In [ ]:
import spacy
from collections import Counter
from string import punctuation
nlp = spacy.load("it_core_news_sm")
def get_hotwords(text):
    result = []
    pos_tag = ['PROPN', 'ADJ', 'NOUN'] 
    doc = nlp(text.lower()) 
    for token in doc:
        if(token.text in nlp.Defaults.stop_words or token.text in punctuation):
            continue
        if(token.pos_ in pos_tag):
            result.append(token.text)
    return result
text = """
Gli audit log devono registrare il verificarsi di un evento, catturando informazioni come l'ora in cui si è verificato, l'utente o il servizio responsabile, es.: o identificativo, descrizione e timestamp dell'evento; o attore (es. ID utente o ID API) e azione eseguita (creazione, modifica, cancellazione, ecc.; o elemento impattato (es. indirizzo IP e identificativo dell’applicazione, dispositivo, sistema, ecc.); o fonte da cui proviene l'attore (paese, nome host, indirizzo IP, ID dispositivo, ecc.); o eventuali tag personalizzati, come il livello di gravità attribuito all'evento (es. clear, warning, minor, major, critical). Asset Management • Onde scongiurare fenomeni di “Shadow IT”, deve essere garantito e documentato l’inserimento, nel’Asset Inventory aziendale, dell’applicazione e dell’ambiente in cui essa opera, di produzione o meno. • L’inserimento nell’Asset Inventory aziendale deve includere il nominativo di chi ha la titolarità (ownership) dell’applicazione / ambiente, ovvero della funzione o ruolo che ha la responsabilità primaria dell’attività, della produttività e della resilienza dell’asset in questione. • Questo principio è essenziale per la risposta alle vulnerabilità e la gestione continua delle stesse, dato che il processo aziendale di risposta a vulnerabilità presenti su un determinato “Asset” risulterebbe bloccato nel caso non fosse preventivamente individuato il relativo “Asset Owner”, cfr. [41].

Linee guida
sullo sviluppo di software sicuro Codice Procedura: LGSWSIC Rev. 03 Pag.
"""
output = set(get_hotwords(text))
most_common_list = Counter(output).most_common(10)
for item in most_common_list:
  print(item[0])

### Yake

In [ ]:

from yake import KeywordExtractor

# Create a KeywordExtractor instance
kw_extractor = KeywordExtractor()

# Text from which keywords will be extracted
text = '''Gli audit log devono registrare il verificarsi di un evento, catturando informazioni come l'ora in cui si è verificato, l'utente o il servizio responsabile, es.: o identificativo, descrizione e timestamp dell'evento; o attore (es. ID utente o ID API) e azione eseguita (creazione, modifica, cancellazione, ecc.; o elemento impattato (es. indirizzo IP e identificativo dell’applicazione, dispositivo, sistema, ecc.); o fonte da cui proviene l'attore (paese, nome host, indirizzo IP, ID dispositivo, ecc.); o eventuali tag personalizzati, come il livello di gravità attribuito all'evento (es. clear, warning, minor, major, critical). Asset Management • Onde scongiurare fenomeni di “Shadow IT”, deve essere garantito e documentato l’inserimento, nel’Asset Inventory aziendale, dell’applicazione e dell’ambiente in cui essa opera, di produzione o meno. • L’inserimento nell’Asset Inventory aziendale deve includere il nominativo di chi ha la titolarità (ownership) dell’applicazione / ambiente, ovvero della funzione o ruolo che ha la responsabilità primaria dell’attività, della produttività e della resilienza dell’asset in questione. • Questo principio è essenziale per la risposta alle vulnerabilità e la gestione continua delle stesse, dato che il processo aziendale di risposta a vulnerabilità presenti su un determinato “Asset” risulterebbe bloccato nel caso non fosse preventivamente individuato il relativo “Asset Owner”, cfr. [41].

Linee guida
sullo sviluppo di software sicuro Codice Procedura: LGSWSIC Rev. 03 Pag.'''

# Extract keywords from the text
keywords = kw_extractor.extract_keywords(text)

# Print the extracted keywords and their scores
for kw in keywords:
    print("Keyword:", kw[0], "Score:", kw[1])

### Rake

In [ ]:
# installation


import nltk
nltk.download('stopwords')
nltk.download('punkt')
from rake_nltk import Rake

# Create a Rake instance
r = Rake()

# Text from which keywords will be extracted
text = '''Gli audit log devono registrare il verificarsi di un evento, catturando informazioni come l'ora in cui si è verificato, l'utente o il servizio responsabile, es.: o identificativo, descrizione e timestamp dell'evento; o attore (es. ID utente o ID API) e azione eseguita (creazione, modifica, cancellazione, ecc.; o elemento impattato (es. indirizzo IP e identificativo dell’applicazione, dispositivo, sistema, ecc.); o fonte da cui proviene l'attore (paese, nome host, indirizzo IP, ID dispositivo, ecc.); o eventuali tag personalizzati, come il livello di gravità attribuito all'evento (es. clear, warning, minor, major, critical). Asset Management • Onde scongiurare fenomeni di “Shadow IT”, deve essere garantito e documentato l’inserimento, nel’Asset Inventory aziendale, dell’applicazione e dell’ambiente in cui essa opera, di produzione o meno. • L’inserimento nell’Asset Inventory aziendale deve includere il nominativo di chi ha la titolarità (ownership) dell’applicazione / ambiente, ovvero della funzione o ruolo che ha la responsabilità primaria dell’attività, della produttività e della resilienza dell’asset in questione. • Questo principio è essenziale per la risposta alle vulnerabilità e la gestione continua delle stesse, dato che il processo aziendale di risposta a vulnerabilità presenti su un determinato “Asset” risulterebbe bloccato nel caso non fosse preventivamente individuato il relativo “Asset Owner”, cfr. [41].

Linee guida
sullo sviluppo di software sicuro Codice Procedura: LGSWSIC Rev. 03 Pag.'''

# Extract keywords from the text
r.extract_keywords_from_text(text)

# Get the ranked keywords
keywords = r.get_ranked_phrases_with_scores()

# Print the extracted keywords and their scores
for score, kw in keywords:
    print("Keyword:", kw, "Score:", score)

### LLM based

In [ ]:
from langchain_ollama import ChatOllama
import json
def keyword_extraction(text:str):
    local_llm = "llama3.2:latest"
    llm_json_mode = ChatOllama(model=local_llm, temperature=0, format="json")
    prompt = f"""Estrai tre keywords da questo testo, cercando di mantenere il significato originale, 
    e dammele in output in formato JSON.
    Le keyword vanno date in output prendendo ad esempio lo schema di questo esempio:
    {{'keywords': ['keyword 1', 'keyword 2', 'keyword 3']}}.
    
    Ricorda questo che hai letto è un esempio e non un output fisso da dare.
    Qui di seguito trovi il testo da cui devi estrarle: 
    {text}
    """

    #prompt_formatted = prompt.format(text=text)
    response = llm_json_mode.invoke(prompt)
    return response



## Retrieval with metadata

In [10]:
import torch
from typing import List
from llama_index.core.node_parser import HierarchicalNodeParser, get_leaf_nodes,get_root_nodes



def create_nodes_with_metadata(docs):
    """
    Crea i nodi partendo da un 'Document' o una lista di Document, 
    aggiunge metadati 'keywords' usando la funzione keyword_extraction.
    Restituisce una tupla: (tutti_i_nodi, nodi_foglia).
    """

    # Parser per creare i nodi
    node_parser = HierarchicalNodeParser.from_defaults(
        chunk_sizes=[512, 1024,2048],
        chunk_overlap=100
    )
    all_nodes = node_parser.get_nodes_from_documents(docs)
    leaf_nodes = get_leaf_nodes(all_nodes)

    # Funzione helper per estrarre le keyword (JSON)
    def get_keywords_from_chunk(chunk_text: str):
        response = keyword_extraction(chunk_text)
        try:
            data = json.loads(response.content)
            # Supponiamo che il JSON abbia la struttura {"keywords": [...]} 
            if isinstance(data, dict) and "keywords" in data:
                return data["keywords"]
            return []
        except json.JSONDecodeError:
            return []

    # Aggiunta dei metadati "keywords" a ciascun nodo foglia
    for node in leaf_nodes:
        chunk_text = node.get_content()
        keywords = get_keywords_from_chunk(chunk_text)
        node.metadata["keywords"] = keywords
        print(node.metadata)

    return all_nodes, leaf_nodes

In [ ]:
nodes,leaf_nodes = create_nodes_with_metadata(docs)

    

In [ ]:
# Salva i nodi in formato JSON
with open('nodes_data.json', 'w', encoding='utf-8') as f:
    # Converti i nodi in dizionari per la serializzazione
    nodes_dict = [node.to_dict() for node in leaf_nodes]
    json.dump(nodes_dict, f, ensure_ascii=False, indent=2)

In [5]:
import json
from llama_index.core.schema import Node

# Carica i nodi dal file JSON
with open('data/nodes_data.json', 'r', encoding='utf-8') as f:
    nodes_dict = json.load(f)
    # Converti i dizionari in oggetti Node
    leaf_2 = [Node.from_dict(node_data) for node_data in nodes_dict]

In [12]:
for idx,node in enumerate(leaf_2):
    leaf_nodes[idx].metadata = node.metadata
    print(leaf_nodes[idx].metadata)


{'keywords': ['Offuscamento dei dati', 'Crittografia', 'Data Protection']}
{'keywords': ['pseudonimizzazione', 'cifratura', 'anonimizzazione']}
{'keywords': ['Riservatezza', 'Crittografia', 'Key Management']}
{'keywords': ['Key Management', 'Crittografia', 'Tecniche di offuscamento dei dati']}
{'keywords': ['pseudonimizzazione', 'dati protetti', 'garante privacy']}
{'keywords': ['Crittografia', 'Anonimizzazione', 'Protezione dei dati']}
{'keywords': ['GDPR', 'anonymisation', 'progettazione sicura']}
{'keywords': ['Privacy by Design', 'Differential Privacy', 'OpenDP']}
{'keywords': ['Anonimizzazione', 'Cifrario', 'Attacco al canale laterale']}
{'keywords': ['criptografia', 'codice segreto', 'confidenzialità']}
{'keywords': ['Crittografia', 'Data Controller', 'Data Processor']}
{'keywords': ['Data Protection', 'Privacy by Design', 'GDPR']}
{'keywords': ['DPO', 'Trattamento dati personali', "Diritto all'oblio"]}
{'keywords': ['Diritto di rettifica', 'De-identificazione', 'DES']}
{'keyword

In [6]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.core.schema import TextNode
import numpy as np
from typing import List, Dict, Any

# Classe per implementare il filtro di similarità personalizzato
class SemanticMetadataFilter:
    def __init__(self, embedding_model=embed_model, similarity_threshold=0.5, top_k=3):
        self.embedding_model = embed_model
        self.similarity_threshold = similarity_threshold
        self.top_k = top_k
        
    def filter_nodes(self, query: str, nodes: List[TextNode]) -> List[TextNode]:
        # Crea embedding della query
        query_embedding = self.embedding_model.get_text_embedding(query)
        
        filtered_nodes = []
        for node in nodes:
            # Estrai keywords dai metadati
            keywords = node.metadata.get("keywords", "")
            if not keywords:
                continue
                
            # Se è una stringa, converti in lista
            if isinstance(keywords, str):
                # Assumendo che le keywords siano separate da virgole o altro delimitatore
                keywords_list = [k.strip() for k in keywords.split(",")]
            else:
                keywords_list = keywords
                
            # Combina le keywords in una singola stringa per l'embedding
            keywords_text = " ".join(keywords_list)
            keywords_embedding = self.embedding_model.get_text_embedding(keywords_text)
            
            # Calcola similarità coseno
            similarity = np.dot(query_embedding, keywords_embedding) / (
                np.linalg.norm(query_embedding) * np.linalg.norm(keywords_embedding)
            )
            
            # Filtra in base alla soglia di similarità
            if similarity >= self.similarity_threshold:
                filtered_nodes.append((node,similarity))

            # Ordina i nodi in base alla similarità
            filtered_nodes = sorted(filtered_nodes, key=lambda x: x[1], reverse=True)
                
        return filtered_nodes[:self.top_k]
    
    def display(self, filtered_nodes: List[tuple]):
        for node in filtered_nodes:
            print(f"Node ID: {node[0].node_id}, Score: {node[1]},  Metadata: {node[0].metadata} \n Content: {node[0].get_content()} \n\n")

        



In [7]:
filtered_nodes = SemanticMetadataFilter().filter_nodes("È stato sviluppato e implementato un programma di sicurezza delle informazioni (compresi i programmi di tutti i settori rilevanti della matrice di controllo del cloud)?", leaf_2)

In [8]:
SemanticMetadataFilter().display(filtered_nodes)

Node ID: e27f03fb-74a3-4966-a0f0-6b1411a06dfb, Score: 0.6610507757834766,  Metadata: {'keywords': ['Superficie di attacco', 'Sicurezza informatica', 'Sviluppo software']} 
 Content:  


Node ID: ab2184c4-aa00-4fc5-9623-9ca587508463, Score: 0.6320444844002139,  Metadata: {'keywords': ['sviluppo software sicuro', 'pianificazione attività', 'requisiti di sicurezza']} 
 Content:  


Node ID: c9132133-1618-4715-a790-8f6db1d568b3, Score: 0.6158680146496079,  Metadata: {'keywords': ['sviluppo sicuro del software', 'sicurezza nel software', 'danni reputazionali']} 
 Content:  




## Sparse retreiver

In [14]:
from llama_index.retrievers.bm25 import BM25Retriever
import Stemmer

# We can pass in the index, docstore, or list of nodes to create the retriever
bm25_retriever = BM25Retriever.from_defaults(
    nodes=nodes,
    similarity_top_k=10,
    # Optional: We can pass in the stemmer and set the language for stopwords
    # This is important for removing stopwords and stemming the query + text
    # The default is english for both
    stemmer=Stemmer.Stemmer("english"),
    language="english",
)

resource module not available on Windows


In [ ]:
from llama_index.core.response.notebook_utils import display_source_node

# will retrieve context from specific companies
retrieved_nodes = bm25_retriever.retrieve(
    "Le politiche e le procedure di gestione della continuità operativa e di resilienza operativa sono stabilite, documentate, approvate, comunicate, applicate a tutti i progetti, valutate e mantenute?"
)
for node in retrieved_nodes:
    display_source_node(node, source_length=10000)

In [ ]:
from llama_index.core.node_parser import SimpleNodeParser
import spacy

# Carica il modello italiano di spaCy
nlp = spacy.load('it_core_news_sm')

# Funzione di preprocessing per l'italiano
def preprocess_text(text):
    doc = nlp(text)
    # Rimuovi stopwords e lemmatizza
    tokens = [token.lemma_.lower() for token in doc 
             if not token.is_stop and not token.is_punct]
    return " ".join(tokens)

# Configura BM25Retriever con preprocessamento italiano
bm25_retriever = BM25Retriever.from_defaults(
    nodes=leaf_nodes,
    tokenizer=lambda x: preprocess_text(x).split(),  # Custom tokenizer
    similarity_top_k=10
)

## Ensamble retreiver

In [ ]:
from llama_index.core.retrievers import BaseRetriever
class EnsembleRetriever(BaseRetriever):
    def __init__(
        self,  
        embedding_model=None, 
        bm25_retriever:BM25Retriever=None, 
        auto_merging_retriever:AutoMergingRetriever=None,
        metadata_retriever:SemanticMetadataFilter=None,
        weights=None
    ):
        
        self.embedding_model = embedding_model
        
        # Inizializza i retrievers se non forniti
        self.bm25_retriever = bm25_retriever 
        
        self.auto_merging_retriever = auto_merging_retriever 
        
        self.metadata_retriever = metadata_retriever 
        
        # Pesi di default se non specificati
        self.weights = weights or {
            'bm25': 0.2,
            'auto_merging': 0.6,
            'metadata': 0.2
        }
    
    def _retrieve(self, query:str):
        # Recupera contesto da ciascun retriever
        bm25_nodes = self.bm25_retriever.retrieve(query)
        auto_merging_nodes = self.auto_merging_retriever.retrieve(query)
        metadata_nodes = self.metadata_retriever.filter_nodes(query)
        
        # Combina i risultati
        combined_nodes = {}
        
        # Funzione per aggiungere nodi con pesatura
        def add_nodes(nodes, weight):
            for node in nodes:
                if node.node.node_id not in combined_nodes:
                    combined_nodes[node.node.node_id] = {
                        'node': node.node,
                        'score': node.score * weight if node.score is not None else weight
                    }
                else:
                    # Aggiorna il punteggio se il nodo esiste già
                    combined_nodes[node.node.node_id]['score'] += node.score * weight if node.score is not None else weight
        
        # Aggiungi nodi da ciascun retriever con i loro pesi
        add_nodes(bm25_nodes, self.weights['bm25'])
        add_nodes(auto_merging_nodes, self.weights['auto_merging'])
        add_nodes(metadata_nodes, self.weights['metadata'])
        
        # Converti in lista ordinata per punteggio
        final_nodes = sorted(
            combined_nodes.values(), 
            key=lambda x: x['score'], 
            reverse=True
        )
        
        # Restituisci i top 10 nodi
        return final_nodes[:10]